# Course 3: Embedded Layer for Natural Language Processing

In this notebook, we will use `PyTorch` to get started with natural language processing. I recommend you to use `spaCy` to do the tokenization and the preprocessing of the text. It is more powerful than the `nltk` library.

## Install the libraries

The `spaCy` library is not installed by default. You can install it by running the following command:

In [1]:
# if using macOS
!pip install 'spacy[transformers,lookups,apple]' torchtext transformers
# if using Windows with GPU
# !pip install -U 'spacy[cuda11x,transformers,lookups]' torchtext transformers
# if using Windows without GPU
# !pip install -U 'spacy[transformers,lookups]' torchtext transformers
# !pip install spacy torchtext transformers
!python -m spacy download en_core_web_sm
!python -m spacy download zh_core_web_sm

Because the competition contains Chinese text, we will use the `zh_core_web_sm` model to tokenize the Chinese text. The `en_core_web_sm` model is used to tokenize the English text.

The `torchtext` library is used to preprocess the text and create the dataloader. It is a powerful library that can help you to preprocess the text and create the dataloader in a few lines of code.

The `transformers` library is used to load the pre-trained model, provided by the Hugging Face team. The pre-trained model is used to convert the text into a vector representation. The vector representation can be used as input to the neural network.

However, it is not recommended to use pre-trained model during the competition. But it is a good practice to use it in the real world project.

Let's start by importing the libraries.

## A small introduction to spaCy

`spaCy` is a powerful library that can help you to tokenize the text. It is more powerful than the `nltk` library. You can vectorize the text and preprocess the text using the `spaCy` library.

To get started, you need to load models, which is downloaded during the installation session. You can load the models by running the following code:

In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")

/opt/anaconda3/envs/training/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/envs/training/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Then, you can tokenize the text (e.g. "I am learning Machine Learning & Natural Language Processing") by running the following code:

In [4]:
doc = nlp("I am learning Machine Learning & Natural Language Processing.")
for token in doc:
    print(token.text, token.pos_, token.dep_)

I PRON nsubj
am AUX aux
learning VERB ROOT
Machine PROPN compound
Learning PROPN dobj
& CCONJ cc
Natural PROPN compound
Language PROPN compound
Processing PROPN conj
. PUNCT punct


As you can see, the `spaCy` library can figure out every part of the text, such as the token, part of speech, and dependency. It is a powerful library that can help you to preprocess the text and create the dataloader.

### Using Chinese

There are numerous differences between English-typical languages and Chinese-typical languages. Chinese doesn't need spaces to split words, but it is harder than English to split the word in Machine Learning. But, don't worry, spaCy helps us to tokenize the Chinese text. You can load the Chinese model by running the following code:

In [7]:
nlp_chinese = spacy.load("zh_core_web_sm")

doc = nlp_chinese("我正在学习机器学习和自然语言处理。")

for token in doc:
    print(token.text, token.pos_, token.dep_)

nanjing_bridge = nlp_chinese("南京市长江大桥是中国南京的一座桥梁。")

for token in nanjing_bridge:
    print(token.text, token.pos_, token.dep_)

我 PRON nsubj
正在 ADV advmod
学习 VERB ROOT
机器 NOUN dobj
学习 VERB conj
和 CCONJ cc
自然 NOUN compound:nn
语言 NOUN compound:nn
处理 NOUN conj
。 PUNCT punct
南京市 PROPN compound:nn
长江 PROPN compound:nn
大桥 NOUN nsubj
是 VERB cop
中国 PROPN name
南京 PROPN nmod:assmod
的 PART case
一 NUM nummod
座 NUM mark:clf
桥梁 NOUN ROOT
。 PUNCT punct


In [2]:
import torch
from torchtext import data, datasets
import spacy

# Load spaCy models
nlp_en = spacy.load("en_core_web_sm")
nlp_zh = spacy.load("zh_core_web_sm")

# Define the fields
TEXT = data.Field(tokenize=lambda x: [tok.text for tok in nlp_en.tokenizer(x)], lower=True)
LABEL = data.LabelField(dtype=torch.float)

# Load IMDb dataset
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split()

/opt/anaconda3/envs/training/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/envs/training/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


The `TEXT` field is used to tokenize the text. The `LABEL` field is used to store the label. The `datasets.IMDB.splits` function is used to load the IMDb dataset. The `train_data` and `test_data` are used to store the training and testing data. The `train_data` is further split into `train_data` and `valid_data`.

### Named entity recognition

`spaCy` can also be used to recognize the named entity in the text. The named entity is the real-world object that is assigned a name. For example, the named entity in the text "Apple is a company" is "Apple". You can recognize the named entity by running the following code:

In [17]:
doc = nlp("Apple is a company, and apple is a fruit. By the way, Apple is located in California.\n Tiananmen Square is located in Beijing, China.")

for ent in doc.ents:
    print(ent.text, ent.label_)

doc = nlp("Zhenhai High School is a high school located in Ningbo, Zhejiang, China. Students in a classroom, learning NLP via Kaggle & GitHub in this class. It cost me 40 minutes, but it is paid for $0.")

for ent in doc.ents:
    print(ent.text, ent.label_)

doc = nlp_chinese("南京市长江大桥是中国南京的一座桥梁。")

for ent in doc.ents:
    print(ent.text, ent.label_)

Apple ORG
Apple ORG
California GPE
Tiananmen Square FAC
Beijing GPE
China GPE
Zhenhai High School ORG
Ningbo GPE
Zhejiang GPE
China GPE
NLP ORG
Kaggle & GitHub ORG
40 minutes TIME
0 MONEY
南京市 GPE
长江大桥 FAC
中国 GPE


Here, the `ent.text` is the named entity, and the `ent.label_` is the type of the named entity. The named entity can be a person, organization, location, product, event, and so on. The named entity recognition is a powerful tool that can help you to extract the information from the text.

### Transplant to Chinese Custom Dataset

In [3]:
# Custom dataset for Chinese (example)
chinese_sentences = [
    ("我今天很开心", "pos"),
    ("这真是糟糕透了", "neg"),
    ("天气不错", "pos"),
    ("我感到很悲伤", "neg")
]

class CustomDataset(data.Dataset):
    def __init__(self, examples, fields, **kwargs):
        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex):
        return len(ex.text)

    @classmethod
    def splits(cls, fields, train_examples=None, val_examples=None, test_examples=None, **kwargs):
        return (cls(train_examples, fields, **kwargs),
                cls(val_examples, fields, **kwargs),
                cls(test_examples, fields, **kwargs))

# Create examples for the custom dataset
chinese_examples = [data.Example.fromlist([sentence, label], [('text', TEXT), ('label', LABEL)]) for sentence, label in chinese_sentences]
chinese_dataset = CustomDataset(chinese_examples, [('text', TEXT), ('label', LABEL)])

# Build the vocabulary
TEXT.build_vocab(train_data, max_size=25000)
LABEL.build_vocab(train_data)

# Add Chinese tokens to the vocabulary
for sentence, _ in chinese_sentences:
    for token in nlp_zh(sentence):
        if token.text not in TEXT.vocab.stoi:
            TEXT.vocab.stoi[token.text] = len(TEXT.vocab.stoi)
            TEXT.vocab.itos.append(token.text)


### Create Iterator

In [4]:
# Create iterators
BATCH_SIZE = 4
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, 
    device=torch.device('mps')
)

chinese_iterator = data.BucketIterator(
    chinese_dataset,
    batch_size=BATCH_SIZE,
    device=torch.device('mps')
)

### Create the Model

Because the situation is quite easy, we can only define a layer of `LSTM` instead of using `transformers` to convert the text into a vector representation. The `LSTM` layer is used to convert the text into a vector representation. The vector representation can be used as input to the neural network.

The `LSTM` layer is a type of recurrent neural network that can be used to process the sequence data. The `LSTM` layer is used to process the text and convert it into a vector representation. The vector representation can be used as input to the neural network.

In [5]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        output, (hidden, cell) = self.rnn(embedded)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        return self.fc(hidden)

# Define hyperparameters
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = SentimentRNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

### Train the Model

In [8]:
import torch.optim as optim
from tqdm import tqdm

device = torch.device('mps')

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    return correct.sum() / len(correct)

def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in tqdm(iterator):
        text = batch.text.to(device)
        label = batch.label.to(device)
        optimizer.zero_grad()
        predictions = model(text).squeeze(1)
        loss = criterion(predictions, label)
        acc = binary_accuracy(predictions, label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in tqdm(iterator):
            text = batch.text.to(device)
            label = batch.label.to(device)
            predictions = model(text).squeeze(1)
            loss = criterion(predictions, label)
            acc = binary_accuracy(predictions, label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# N_EPOCHS = 5

# for epoch in range(N_EPOCHS):
#     train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
#     valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
#     print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')

# Load the state dict
model.load_state_dict(torch.load('./data/03-state_dict.pth'))

<All keys matched successfully>

### Test the Model

In [9]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc*100:.2f}%')

 97%|█████████▋| 6054/6250 [02:00<00:03, 50.08it/s] 


RuntimeError: MPS backend out of memory (MPS allocated: 120.00 MB, other allocations: 18.01 GB, max allowed: 18.13 GB). Tried to allocate 6.22 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

### Predict the Sentiment

In [11]:
def predict_sentiment(model, sentence, nlp, min_len=5):
    model.eval()
    tokens = [tok.text for tok in nlp(sentence)]
    if len(tokens) < min_len:
        tokens += ['<pad>'] * (min_len - len(tokens))
    indexed = [TEXT.vocab.stoi[t] for t in tokens]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    if prediction.item() >= 0.5:
        return 'pos'
    else:
        return 'neg'

# English sentence
print(predict_sentiment(model, "This movie so great!", nlp_en))
# Chinese sentence
print(predict_sentiment(model, "我不高兴……", nlp_en))

neg
neg


### Save the Model

In [ ]:
torch.save(model.state_dict(), './data/03-state_dict.pth')
torch.save(model, './data/03-model.pth')

### Play with the Model

In [12]:
while True:
    sentence = input("Enter a sentence: ")
    if sentence == 'exit':
        break
    print(predict_sentiment(model, sentence, nlp_en))

neg
pos
pos
neg
neg
neg
neg
